Чтение ODF-файлов и извлечение сырых данных
summary (метаданные), orbit (доплер), ramp (частотные настройки)

In [2]:
import os
import pandas as pd
import pdr

def bytes_to_int(v):
    if isinstance(v, int):
        return v
    if isinstance(v, float):
        return int(v)
    if isinstance(v, bytes):
        return int.from_bytes(v, "big", signed=False)
    if isinstance(v, str):
        return int.from_bytes(v.encode("latin1"), "big", signed=False)
    return 0

odf_folder = "input_data"
odf_files = [f for f in os.listdir(odf_folder) if f.endswith(".odf")]

all_summary = []
all_orbit = []
all_ramp = []

ramp_ids = [15, 45, 54, 65]

for odf_file in odf_files:
    odf_path = os.path.join(odf_folder, odf_file)
    lbl_file = odf_file.replace(".odf", ".lbl")
    lbl_path = os.path.join(odf_folder, lbl_file)

    odf_data = pdr.read(odf_path, lbl_path)

    # Data Summary Group DATA (основная сводка наблюдений)
    # Когда, какая станция, сколько сэмплов
    if "ODF7B_TABLE" in odf_data: 
        table = odf_data["ODF7B_TABLE"]
        for row in table:
            if len(row) < 9:
                continue
            obs = {
                "t_start_sec": bytes_to_int(row[0]),
                "t_start_frac": bytes_to_int(row[1]),
                "station_id": bytes_to_int(row[2]),
                "doppler_channel": bytes_to_int(row[3]),
                "band_id": bytes_to_int(row[4]),
                "data_type": bytes_to_int(row[5]),
                "n_samples": bytes_to_int(row[6]),
                "t_end_sec": bytes_to_int(row[7]),
                "t_end_frac": bytes_to_int(row[8])
            }
            all_summary.append(obs)

    # Orbit Data Group DATA (основные телекоммуникационные наблюдения)
    # Доплер, время, фракции
    if "ODF3C_TABLE" in odf_data: 
        table = odf_data["ODF3C_TABLE"]
        for row in table:
            if len(row) < 4:
                continue

            time_sec = bytes_to_int(row[0])
            frac_bits = bytes_to_int(row[1])
            observable_int = bytes_to_int(row[2])
            observable_frac = bytes_to_int(row[3])

            orbit = {
                "time_sec": time_sec,
                "time_frac_ms": frac_bits & 0x3FF,
                "downlink_delay_ns": (frac_bits >> 10) & 0x3FFFFF,
                "observable_int": observable_int,
                "observable_frac": observable_frac
            }
            all_orbit.append(orbit)

    # Ramp Group x DATA (частотные настройки трансиверов)
    # Настройка частот на станциях DSN №x
    for x in ramp_ids:
        table_name = f"ODF4B{x}_TABLE"
        if table_name not in odf_data:
            continue

        table = odf_data[table_name]
        for row in table:
            if len(row) < 9:
                continue

            ramp = {
                "ramp_group_id": x,
                "ramp_start_sec": bytes_to_int(row[0]),
                "ramp_start_frac": bytes_to_int(row[1]),
                "ramp_rate_int": bytes_to_int(row[2]),
                "ramp_rate_frac": bytes_to_int(row[3]),
                "station_id": bytes_to_int(row[4]) & 0x3FF,
                "ramp_start_freq_int": bytes_to_int(row[5]),
                "ramp_start_freq_frac": bytes_to_int(row[6]),
                "ramp_end_sec": bytes_to_int(row[7]),
                "ramp_end_frac": bytes_to_int(row[8])
            }
            all_ramp.append(ramp)


pd.DataFrame(all_summary).to_csv("mgs_moi_odf7b.csv", index=False, sep=';')
pd.DataFrame(all_orbit).to_csv("mgs_moi_odf3c.csv", index=False, sep=';')
pd.DataFrame(all_ramp).to_csv("mgs_moi_odf4b_all.csv", index=False, sep=';')

print("Summary записей:", len(all_summary))
print("Orbit записей:", len(all_orbit))
print("Ramp записей:", len(all_ramp))


Summary записей: 64
Orbit записей: 48
Ramp записей: 189


Обработка ODF: интерполяция ramp и построение таблицы Доплера
Для каждого наблюдения Доплера рассчитывается передающая частота с учётом ramp → данные готовы для расчёта теоретического Доплера

In [13]:
import os
import numpy as np
import pandas as pd
import pdr 

def bytes_to_int(v):
    """Конвертирует разные типы данных в целое число"""
    if isinstance(v, int):
        return v
    if isinstance(v, float):
        return int(v)
    if isinstance(v, (bytes, bytearray)):
        return int.from_bytes(v, "big", signed=False)
    if isinstance(v, str):
        return int.from_bytes(v.encode("latin1"), "big", signed=False)
    return 0

def parse_odf_folder(odf_folder, ramp_ids=[15,45,54,65]):
    """Парсит ODF-файлы: summary, доплер и ramp"""
    summary = []
    doppler = []
    ramp = []
    
    odf_files = [f for f in os.listdir(odf_folder) if f.lower().endswith(".odf")]
    for odf_file in odf_files:
        lbl_file = odf_file[:-4] + ".lbl"
        odf_path = os.path.join(odf_folder, odf_file)
        lbl_path = os.path.join(odf_folder, lbl_file)
        try:
            data = pdr.read(odf_path, lbl_path)
        except Exception as e:
            print("ERROR reading", odf_file, e)
            continue
        
        # --- Summary (метаданные наблюдений) ---
        if "ODF7B_TABLE" in data:
            for row in data["ODF7B_TABLE"]:
                if len(row) < 9:
                    continue
                obs = {
                    "t_start_sec": bytes_to_int(row[0]),
                    "t_start_frac": bytes_to_int(row[1]),
                    "station_id": bytes_to_int(row[2]),
                    "doppler_channel": bytes_to_int(row[3]),
                    "band_id": bytes_to_int(row[4]),
                    "data_type": bytes_to_int(row[5]),
                    "n_samples": bytes_to_int(row[6]),
                    "t_end_sec": bytes_to_int(row[7]),
                    "t_end_frac": bytes_to_int(row[8]),
                }
                summary.append(obs)
        
        # --- Doppler / orbit data ---
        if "ODF3C_TABLE" in data:
            for row in data["ODF3C_TABLE"]:
                if len(row) < 4:
                    continue
                time_sec = bytes_to_int(row[0])
                frac = bytes_to_int(row[1])
                obs_int = bytes_to_int(row[2])
                obs_frac = bytes_to_int(row[3])
                
                # объединяем целую и дробную часть
                dop = obs_int + obs_frac / (2**32)
                
                # конвертируем время + дробь в секунды
                time = time_sec + frac / 1024.0 
                
                doppler.append({
                    "time": time,
                    "doppler_count": dop,
                })
        
        # --- Ramp tables ---
        for rid in ramp_ids:
            table_name = f"ODF4B{rid}_TABLE"
            if table_name not in data:
                continue
            for row in data[table_name]:
                if len(row) < 9:
                    continue
                ramp.append({
                    "ramp_group_id": rid,
                    "ramp_start_sec": bytes_to_int(row[0]),
                    "ramp_start_frac": bytes_to_int(row[1]),
                    "ramp_rate_int": bytes_to_int(row[2]),
                    "ramp_rate_frac": bytes_to_int(row[3]),
                    "station_id": bytes_to_int(row[4]) & 0x3FF,
                    "ramp_start_freq_int": bytes_to_int(row[5]),
                    "ramp_start_freq_frac": bytes_to_int(row[6]),
                    "ramp_end_sec": bytes_to_int(row[7]),
                    "ramp_end_frac": bytes_to_int(row[8]),
                })
    
    df_summary = pd.DataFrame(summary)
    df_doppler = pd.DataFrame(doppler)
    df_ramp = pd.DataFrame(ramp)
    return df_summary, df_doppler, df_ramp


def interpolate_ramp(df_ramp, times):
    """
    Интерполирует передающую частоту для каждого времени наблюдения
    с учётом ramp.
    """
    out = []
    for t in times:
        df_sel = df_ramp[(df_ramp["ramp_start_sec"] + df_ramp["ramp_start_frac"]/1024.0) <= t]
        if df_sel.empty:
            out.append(np.nan)
        else:
            last = df_sel.iloc[-1]
            t0 = last["ramp_start_sec"] + last["ramp_start_frac"]/1024.0
            dt = t - t0
            f0 = last["ramp_start_freq_int"] + last["ramp_start_freq_frac"]/(2**32)
            rate = last["ramp_rate_int"] + last["ramp_rate_frac"]/(2**32)
            f = f0 + rate * dt
            out.append(f)
    return np.array(out)


def build_final_table(odf_folder, output_csv="mgs_doppler_ramp.csv"):
    """Создаёт финальную таблицу Доплера с интерполированными частотами"""
    df_s, df_d, df_r = parse_odf_folder(odf_folder)
    df_d = df_d.sort_values("time").reset_index(drop=True)
    df_d["tx_freq"] = interpolate_ramp(df_r, df_d["time"].values)
    df_d.to_csv(output_csv, index=False, sep=';')
    print("Wrote", output_csv, "lines:", len(df_d))


if __name__ == "__main__":
    odf_folder = "input_data"
    build_final_table(odf_folder, output_csv="mgs_doppler_ramp.csv")


Wrote mgs_doppler_ramp.csv lines: 48


Проверка данных:

In [14]:
import pandas as pd

df = pd.read_csv("mgs_doppler_ramp.csv", sep=';')
print(df.head())
print(df.describe())

        time  doppler_count  tx_freq
0  73.082031           69.0     32.0
1  73.082031           69.0     32.0
2  73.082031           69.0     32.0
3  73.082031           69.0     32.0
4  73.082031           69.0     32.0
            time  doppler_count     tx_freq
count  48.000000      48.000000   48.000000
mean   76.907715      75.000000  201.529297
std     4.218044       6.391484  186.824054
min    73.082031      69.000000   32.000000
25%    73.082031      69.000000   32.000000
50%    76.073242      73.000000  127.800781
75%    79.064453      83.000000  444.787109
max    84.071289      83.000000  444.787109


Загружает таблицу Доплера, преобразует время в UTC, получает эпемериды Марса, Земли и Солнца через JPL Horizons и приводит все времена к одной системе для анализа.

In [ ]:
import pandas as pd
from astroquery.jplhorizons import Horizons
from datetime import datetime, timedelta

# Загружаем CSV с доплером
df_doppler = pd.read_csv("mgs_doppler_ramp.csv", sep=';')
df_doppler = df_doppler.sort_values("time").reset_index(drop=True)

# Преобразуем время доплера в "абсолютное" UTC
start_date = datetime(2025, 11, 30, 0, 0, 0)
df_doppler['datetime_utc'] = df_doppler['time'].apply(lambda t: start_date + timedelta(seconds=t))

# Запрашиваем эпемериды через JPL Horizons
def get_ephemeris(target_id, location_id, start_date, end_date, step='1d'):
    obj = Horizons(
        id=target_id,
        location=location_id,
        epochs={'start': start_date, 'stop': end_date, 'step': step}
    )
    eph = obj.vectors()
    df = eph.to_pandas()
    return df

df_mars = get_ephemeris('499', '399', start_date.strftime('%Y-%m-%d'),
                        (start_date + timedelta(days=5)).strftime('%Y-%m-%d'), step='1d')
df_earth = get_ephemeris('399', '@sun', start_date.strftime('%Y-%m-%d'),
                         (start_date + timedelta(days=5)).strftime('%Y-%m-%d'), step='1d')
df_sun = get_ephemeris('@0', '@sun', start_date.strftime('%Y-%m-%d'),
                       (start_date + timedelta(days=5)).strftime('%Y-%m-%d'), step='1d')

# Приводим времена к одной системе
for df in [df_mars, df_earth, df_sun]:
    # Убираем префикс "A.D. " и лишние дробные секунды
    df['datetime_utc'] = pd.to_datetime(df['datetime_str'].str.replace("A.D. ", "").str[:19],
                                        format='%Y-%b-%d %H:%M:%S')

# Проверка
print("Doppler sample:")
print(df_doppler.head())

print("\nMars ephemeris sample:")
print(df_mars.head())


Doppler sample:
        time  doppler_count  tx_freq               datetime_utc
0  73.082031           69.0     32.0 2025-11-30 00:01:13.082031
1  73.082031           69.0     32.0 2025-11-30 00:01:13.082031
2  73.082031           69.0     32.0 2025-11-30 00:01:13.082031
3  73.082031           69.0     32.0 2025-11-30 00:01:13.082031
4  73.082031           69.0     32.0 2025-11-30 00:01:13.082031

Mars ephemeris sample:
   targetname  datetime_jd                    datetime_str         x  \
0  Mars (499)    2461009.5  A.D. 2025-Nov-30 00:00:00.0000 -0.495595   
1  Mars (499)    2461010.5  A.D. 2025-Dec-01 00:00:00.0000 -0.464865   
2  Mars (499)    2461011.5  A.D. 2025-Dec-02 00:00:00.0000 -0.434017   
3  Mars (499)    2461012.5  A.D. 2025-Dec-03 00:00:00.0000 -0.403056   
4  Mars (499)    2461013.5  A.D. 2025-Dec-04 00:00:00.0000 -0.371988   

          y         z        vx        vy        vz  lighttime     range  \
0 -2.372460 -0.027611  0.030561 -0.006265 -0.000418   0.013999  2.4

Интерполяция позиций и скоростей Марса, Земли и Солнца на моменты наблюдений Доплера

In [17]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d


def interp_ephem(df_ephem, times, columns=['x','y','z','vx','vy','vz']):
    """
    Линейная интерполяция эфемерид на заданные времена.
    df_ephem: DataFrame с колонкой 'datetime_utc' и колонками positions/velocities
    times: массив datetime (np.datetime64 или pd.Timestamp)
    columns: какие колонки интерполировать
    """
    t_orig = df_ephem['datetime_utc'].astype(np.int64)  # время в наносекундах
    t_target = np.array(times).astype(np.int64)
    
    result = {}
    for col in columns:
        f = interp1d(t_orig, df_ephem[col].values, kind='linear', fill_value='extrapolate')
        result[col] = f(t_target)
    return pd.DataFrame(result, index=times)

# Интерполируем позиции и скорости
df_mars_interp = interp_ephem(df_mars, df_doppler['datetime_utc'])
df_earth_interp = interp_ephem(df_earth, df_doppler['datetime_utc'])
df_sun_interp = interp_ephem(df_sun, df_doppler['datetime_utc'])

# Добавим к Doppler
df_doppler['mars_x'] = df_mars_interp['x'].values
df_doppler['mars_y'] = df_mars_interp['y'].values
df_doppler['mars_z'] = df_mars_interp['z'].values
df_doppler['mars_vx'] = df_mars_interp['vx'].values
df_doppler['mars_vy'] = df_mars_interp['vy'].values
df_doppler['mars_vz'] = df_mars_interp['vz'].values

df_doppler['earth_x'] = df_earth_interp['x'].values
df_doppler['earth_y'] = df_earth_interp['y'].values
df_doppler['earth_z'] = df_earth_interp['z'].values
df_doppler['earth_vx'] = df_earth_interp['vx'].values
df_doppler['earth_vy'] = df_earth_interp['vy'].values
df_doppler['earth_vz'] = df_earth_interp['vz'].values

df_doppler['sun_x'] = df_sun_interp['x'].values
df_doppler['sun_y'] = df_sun_interp['y'].values
df_doppler['sun_z'] = df_sun_interp['z'].values
df_doppler['sun_vx'] = df_sun_interp['vx'].values
df_doppler['sun_vy'] = df_sun_interp['vy'].values
df_doppler['sun_vz'] = df_sun_interp['vz'].values

print(df_doppler.head())
df_doppler.to_csv("mgs_doppler_ramp_interp.csv", index=False, sep=';')

        time  doppler_count  tx_freq               datetime_utc    mars_x  \
0  73.082031           69.0     32.0 2025-11-30 00:01:13.082031 -0.495569   
1  73.082031           69.0     32.0 2025-11-30 00:01:13.082031 -0.495569   
2  73.082031           69.0     32.0 2025-11-30 00:01:13.082031 -0.495569   
3  73.082031           69.0     32.0 2025-11-30 00:01:13.082031 -0.495569   
4  73.082031           69.0     32.0 2025-11-30 00:01:13.082031 -0.495569   

     mars_y    mars_z   mars_vx   mars_vy   mars_vz  ...   earth_z  earth_vx  \
0 -2.372465 -0.027611  0.030561 -0.006265 -0.000418  ... -0.000057 -0.016193   
1 -2.372465 -0.027611  0.030561 -0.006265 -0.000418  ... -0.000057 -0.016193   
2 -2.372465 -0.027611  0.030561 -0.006265 -0.000418  ... -0.000057 -0.016193   
3 -2.372465 -0.027611  0.030561 -0.006265 -0.000418  ... -0.000057 -0.016193   
4 -2.372465 -0.027611  0.030561 -0.006265 -0.000418  ... -0.000057 -0.016193   

   earth_vy      earth_vz     sun_x     sun_y     sun_z 